In [1]:
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
% matplotlib inline

In [2]:
classes = {0: 'CubeBody',  1:'CubeFace', 2: 'CylinderBody',3:  'CylinderFace' }

In [3]:
classes.keys()[list(classes.values()).index("CubeFace")]

1

In [4]:
fcis_path = '/home/icl/chenxin/onekeyGeonet/Evaluations/MeanAP/Exper_FCIS_DCN/10_30_FCIS_ob/'
folderpath = 'compbnf/'
our_path = '/home/icl/chenxin/onekeyGeonet/Evaluations/MeanAP/Exper_FCIS_DCN/10_30_FCIS_DCN/'
bnf_path = folderpath + 'new_bnf/'
gt_path = '/home/icl/chenxin/onekeyGeonet/Evaluations/MeanAP/Exper_FCIS_DCN/gt/'
our_list = os.listdir(our_path)
bnf_list = os.listdir(bnf_path)
gt_list = os.listdir(gt_path)

In [5]:
def mask_overlap_inun(mask11, mask22):
###############################################
###############################################
    """
    This function calculate region IOU when masks are
    inside different boxes
    Returns:
        intersection over unions of this two masks
    """
    mask1 = mask11.copy()
    mask2 = mask22.copy()
    
    mask1[mask1>=1] = 1
    mask2[mask2>=1] = 1
    
    #plt.subplot(1,2,1) 
    #plt.imshow(mask1)
    #plt.subplot(1,2,2) 
    #plt.imshow(mask2)
    
    green = (255,255,255)
    #box1 / box2 are the bounding box of mask1 / mask2 
    blob1 = np.where(mask1>=1)
    box1x1 = min(blob1[1])
    box1x2 = max(blob1[1])
    box1y1 = min(blob1[0])
    box1y2 = max(blob1[0])
    
    #cv2.rectangle(mask1,(box1x1,box1y1),(box1x2,box1y2),0.5,1)
    
    #plt.subplot(1,2,1) 
    #plt.imshow(mask1)
    #plt.subplot(1,2,2) 
    #plt.imshow(mask2)
    
    #print box1x1,box1x2,box1y1,box1y2
    
    blob2 = np.where(mask2>=1)
    box2x1 = min(blob2[1])
    box2x2 = max(blob2[1])
    box2y1 = min(blob2[0])
    box2y2 = max(blob2[0])
    
    #cv2.rectangle(mask2,(box2x1,box2y1),(box2x2,box2y2),0.5,1)
    
    x1 = max(box1x1 , box2x1)
    y1 = max(box1y1 , box2y1)
    x2 = min(box1x2 , box2x2)
    y2 = min(box1y2 , box2y2)

    #######################
    
    if x1 > x2 or y1 > y2:
        return 0
    w = x2 - x1 + 1
    h = y2 - y1 + 1
    # get masks in the intersection part
    inter_maska = mask1[y1: y1 + h, x1:x1 + w]
    inter_maskb = mask2[y1: y1 + h, x1:x1 + w]

    #print inter_maska
    #print inter_maskb
    
    #plt.subplot(2,2,1) 
    #plt.imshow(inter_maska)
    #plt.subplot(2,2,2) 
    #plt.imshow(inter_maskb)    
    
    assert inter_maska.shape == inter_maskb.shape
    inter = np.logical_and(inter_maskb, inter_maska).sum()
    #print inter_maskb
    #print inter_maska
    union = mask1.sum() + mask2.sum() - inter
    
    if union < 1.0:
        return 0
    return float(inter), float(union)

In [6]:
def mask_overlap(mask11, mask22):
    inter,union = mask_overlap_inun(mask11,mask22)
    return float(inter) / float(union)


In [7]:
def ClassifyColor(Red,Green,Blue):
    if Blue == 255:
        if Green == 0 and Red % 10 == 0 and Red != 0:
            return 'CubeBody'
        elif Red % 10 == 0 and Green == Red and Red != 0:
            return 'CubeFace'
    elif Blue == 200:
        if Green == 0 and Red % 10 == 0 and Red != 0:
            return 'CylinderBody'
        elif Red % 10 == 0 and Green == Red and Red != 0:
            return 'CylinderFace'
    elif Blue == 150:
        if Green == 0 and Red % 10 == 0 and Red != 0:
            #return 'Handle'
            return 'CylinderBody'
    elif Red == 53 and Green == 42 and Blue == 135:
            return 'Background'
    elif Red == 54 and Green == 55 and Blue == 160:
            return 'CylinderFace'
    elif Red == 54 and Green == 48 and Blue == 147:
            return 'CylinderBody'   
    elif Red == 50 and Green == 67 and Blue == 186:
            return 'CubeFace'
    elif Red == 53 and Green == 61 and Blue == 173:
            return 'CubeBody'  
    elif Red == 0 and Green == 0 and Blue == 0:
            return 'Background'
    return 'Valid'

In [8]:
def ParseMask(img,h,w):
    part = []
    visited_color = []
    part_label = []
    blue_channel = img[:,:,0]
    pixel = blue_channel.nonzero()
    for i in range(0,len(pixel[0])):
        color = img[pixel[0][i],pixel[1][i]]
        color_string =  str("%03d" % color[0]) + str("%03d" % color[1]) +str("%03d" % color[2])
        if color_string not in visited_color: 
            visited_color.append(color_string)
            label = ClassifyColor(color[2],color[1],color[0])
            #print(label)
            if label!= 'Background':
                lower = np.array(color, dtype = "uint8")#颜色下限  
                upper = np.array(color, dtype = "uint8")#颜色上限  
                part_label.append(label)
                #plt.imshow(fcis_img)
                mask = cv2.inRange(img, lower, upper) 
                output = cv2.bitwise_and(img,img,mask=mask)
                output = cv2.cvtColor(output, cv2.COLOR_BGR2GRAY)
                output[output>=1] = 1

                part.append(output)    
    
    merged_part = []
    merged_label = []
    marked_list = np.zeros(len(part))
    for i in range(0,len(part)):
        if marked_list[i]==1:
            continue
        new_mask = part[i]
        new_mask[new_mask>=1] = 1
        marked_list[i] = 1
        for j in range(0,len(part)):
            if part_label[j] == part_label[i] and marked_list[j]==0 and i!=j:
                new_mask = np.logical_or(new_mask,part[j])
                new_mask[new_mask>=1] = 1
                marked_list[j] = 1
        merged_part.append(new_mask)
        merged_label.append(part_label[i])
    return merged_part,merged_label

In [25]:
def SetMask(img,h,w):
    marked_img = np.zeros([h,w], np.int8)
    for i in range(0,h):
        for j in range(0,w):
            color = img[i,j]
            label = ClassifyColor(color[2],color[1],color[0])
            if label != 'Background':
                cls = classes.keys()[list(classes.values()).index(label)]
                marked_img[i,j] = cls+1
    #plt.imshow(marked_img)    
    return marked_img

In [48]:
def GetOverLap_inun(part,gt_part,part_label,gt_part_label):
    ov_cls_inter = np.zeros(len(classes))
    ov_cls_union = np.zeros(len(classes))
    
    for i in range(0,len(part)):
        cur_overlap = -1000
        cur_overlap_ind = -1
        cur_inter = 0
        cur_union = 0
        for j in range(0,len(gt_part)):
            if part_label[i]== gt_part_label[j]:
                inter,union = mask_overlap_inun(gt_part[j], part[i])
                ov = inter / union
                if ov > cur_overlap:
                    cur_overlap = ov
                    cur_overlap_ind = j       
                    cur_inter = inter
                    cur_union = union
        if cur_overlap == -1000:
            print('overlap==-1000')
            cur_overlap = 0
        
        cls = classes.keys()[list(classes.values()).index(part_label[i])]
        
        ov_cls_inter[cls] =  ov_cls_inter[cls] + cur_inter
        ov_cls_union[cls] = ov_cls_union[cls]+cur_union
        
    return ov_cls_inter,ov_cls_union

In [45]:
def GetOverLap(part,gt_part,part_label,gt_part_label):
    ov_cls = np.zeros(len(classes))
    ov_cls_counter =np.zeros(len(classes))
    
    #ov_areas = np.zeros(len(part))    
    #match_gt_id = np.zeros(len(part))
    
    for i in range(0,len(part)):
        cur_overlap = -1000
        cur_overlap_ind = -1
        for j in range(0,len(gt_part)):
            if part_label[i]== gt_part_label[j]:
                ov = mask_overlap(gt_part[j], part[i])
                if ov > cur_overlap:
                    cur_overlap = ov
                    cur_overlap_ind = j                    
        if cur_overlap == -1000:
            print('overlap==-1000')
            cur_overlap = 0
        
        cls = classes.keys()[list(classes.values()).index(part_label[i])]
        ov_cls[cls] = ov_cls[cls]+cur_overlap
        ov_cls_counter[cls] = ov_cls_counter[cls]  +1
        #ov_areas[i]=cur_overlap
        #match_gt_id[i] = cur_overlap_ind
    mean_ov = np.zeros(len(classes))
    for k in range(0,len(classes)):
        if ov_cls_counter[k]!=0:
            mean_ov[k] = ov_cls[k]/ov_cls_counter[k]
        
    return mean_ov

In [27]:
def mask_level_acc(img,gtimg):
    correctpix = np.zeros(len(classes))
    allpix = np.zeros(len(classes))
    h = img.shape[0]
    w = img.shape[1]
    for i in range(0,h):
        for j in range(0,w):
            pixel = img[i,j]-1
            gt = gtimg[i,j]-1
            if pixel != -1:
                if pixel == gt:
                    correctpix[pixel] = correctpix[pixel] +1
    
    #mean_acc = np.zeros(len(classes))
    #for k in range(0,len(classes)):
     #   if allpix[k]!=0:
      #      mean_acc[k] = correctpix[k]/allpix[k]  
    return correctpix,allpix

In [33]:
##########build gt
gt_img_all = []
gt_part_all = []
gt_part_label_all = []
idx = 0

boxc = 0
cyc = 0

for imgname in bnf_list:
    realname = imgname[:imgname.index('_bnf')]  
    print(str(idx) + " " + imgname)     
    gt_img = cv2.imread(gt_path+realname+'.png', 1)
    
    h = gt_img.shape[0]
    w = gt_img.shape[1]        
        
    #gt_img_l = SetMask(gt_img,h,w)
    gt_part,gt_part_label = ParseMask(gt_img,h,w)
    
    for x in gt_part_label:
        if x == 'CylinderBody':
            cyc = cyc+1
        if x == 'CubeBody':
            boxc = boxc +1
    
    #gt_img_all.append(gt_img_l)
    gt_part_all.append(gt_part)
    gt_part_label_all.append(gt_part_label)
    
    idx = idx+1

0 cube_00651_bnf.png
1 cub_01589_bnf.png
2 cup_00320_bnf.png
3 bow_00098_bnf.png
4 vas_00093_bnf.png
5 cylinder_02334_bnf.png
6 cylinder_01354_bnf.png
7 cube_00030_bnf.png
8 cub_00351_bnf.png
9 cube_00327_bnf.png
10 bla_00170_bnf.png
11 bow_00147_bnf.png
12 cylinder_02452_bnf.png
13 cub_00777_bnf.png
14 cub_01320_bnf.png
15 bla_00186_bnf.png
16 cup_00207_bnf.png
17 cube_00612_bnf.png
18 cylinder_01171_bnf.png
19 bow_00031_bnf.png
20 cylinder_01303_bnf.png
21 cylinder_01222_bnf.png
22 cube_01541_bnf.png
23 cube_00574_bnf.png
24 bla_00187_bnf.png
25 cub_01132_bnf.png
26 cylinder_01204_bnf.png
27 cube_00459_bnf.png
28 bow_00102_bnf.png
29 bla_00148_bnf.png
30 cube_01724_bnf.png
31 cube_00706_bnf.png
32 cube_00194_bnf.png
33 lov_00255_bnf.png
34 cub_00474_bnf.png
35 cup_00104_bnf.png
36 jui_00079_bnf.png
37 vas_00224_bnf.png
38 cube_00655_bnf.png
39 cube_00464_bnf.png
40 vas_00257_bnf.png
41 cub_00257_bnf.png
42 cylinder_01984_bnf.png
43 cube_00228_bnf.png
44 cup_00531_bnf.png
45 cylinder_

In [34]:
print(cyc)
print(boxc)

174
165


In [49]:
###################PP-IOU
idx= 0
test_acc_all = []

test_correct = np.zeros(len(classes))
test_all = np.zeros(len(classes))

for imgname in bnf_list:
    realname = imgname[:imgname.index('_bnf')]   
    test_img = cv2.imread(bnf_path+realname+'_bnf.png', 1)
    
    print(str(idx) + " " + imgname) 
    
    if test_img is None:
        print("no test output")
        test_img =  np.zeros(gt_img_all[idx].shape,np.uint8)

    h = test_img.shape[0]
    w = test_img.shape[1]          
        
    test_img = SetMask(test_img,h,w)    
    correct, allpix = mask_level_acc(test_img,gt_img_all[idx])
      
    for k in range(0,len(classes)):
        test_correct[k] = test_correct[k]  + correct[k]
        test_all[k] = test_all[k]  + allpix[k]    
    
    idx = idx+1
      
test_acc_all = np.zeros(len(classes))
for k in range(0,len(classes)):
    if test_all[k]!=0:
        test_acc_all[k] = test_correct[k]/test_all[k]
        


0 cube_00651_bnf.png


IndexError: list index out of range

In [394]:
print(test_acc_all)
print(test_acc_all.mean())

[ 0.97890348  0.96903999  0.93847513  0.96186267]
0.962070315316


In [ ]:
#############################################################################################################

In [82]:
###############NEW-PP-IOU
idx= 0
test_part_all = []
test_part_label_all = []

for imgname in bnf_list:
    realname = imgname[:imgname.index('_bnf')]   
    test_img = cv2.imread(bnf_path+realname+'_bnf.png', 1)
    #print(str(idx) + " " + imgname) 

    h = test_img.shape[0]
    w = test_img.shape[1]
    
    if test_img is None:
        print("no test output")
        test_img =  np.zeros(gt_img_all[idx].shape,np.uint8)
             
    test_part,test_part_label = ParseMask(test_img,h,w)  
    test_part_all.append(test_part)    
    test_part_label_all.append(test_part_label)
    
    idx = idx+1

test_in_all = []
test_un_all = []
for x in range(0,len(test_part_all)):
    #print(x)
    test_in,test_un = GetOverLap_inun(test_part_all[x],gt_part_all[x],test_part_label_all[x],gt_part_label_all[x])    
    #print(test_ov)
    test_in_all.append(test_in)
    test_un_all.append(test_un)

overlap==-1000
overlap==-1000
overlap==-1000
overlap==-1000


In [83]:
###########################PP-IOU
test_in_all = np.array(test_in_all)
test_un_all = np.array(test_un_all)

print(test_in_all.sum(0))
print(test_un_all.sum(0))
print('-----')
print(test_in_all.sum(0)/test_un_all.sum(0))
print((test_in_all.sum(0)/test_un_all.sum(0)).mean())

[  5791021.   2858626.  10270405.   3595636.]
[  6123496.   3056577.  11399328.   4366284.]
-----
[ 0.94570504  0.93523769  0.90096583  0.82350026]
0.901352201932


In [84]:
########################PI-IOU
test_ov_all_np = np.zeros(len(classes))
test_ov_all_counter = np.zeros(len(classes))
for k in range(0,len(test_un_all)):   
    for j in range(0,len(classes)):
        if test_un_all[k][j]!=0:
            test_ov_all_np[j] = test_ov_all_np[j] + test_in_all[k][j]/test_un_all[k][j]
            test_ov_all_counter[j] = test_ov_all_counter[j] + 1
print(test_ov_all_np)
print(test_ov_all_counter)
print('----')
print(test_ov_all_np/test_ov_all_counter)
print((test_ov_all_np/test_ov_all_counter).mean())

[ 153.73657612  148.2506975   153.93289938  119.70670497]
[ 165.  162.  174.  167.]
----
[ 0.93173682  0.91512776  0.88467184  0.71680662]
0.862085759852


In [ ]:
#############################################################################################################

In [40]:
###############PI-IOU
idx= 0
test_part_all = []
test_part_label_all = []

for imgname in bnf_list:
    realname = imgname[:imgname.index('_bnf')]   
    test_img = cv2.imread(fcis_path+realname+'.png', 1)
    print(str(idx) + " " + imgname) 

    h = test_img.shape[0]
    w = test_img.shape[1]
    
    if test_img is None:
        print("no test output")
        test_img =  np.zeros(gt_img_all[idx].shape,np.uint8)
             
    test_part,test_part_label = ParseMask(test_img,h,w)  
    test_part_all.append(test_part)    
    test_part_label_all.append(test_part_label)
    
    idx = idx+1

test_ov_all = []
for x in range(0,len(test_part_all)):
    #print(x)
    test_ov = GetOverLap(test_part_all[x],gt_part_all[x],test_part_label_all[x],gt_part_label_all[x])    
    #print(test_ov)
    
    test_ov_all.append(test_ov)

0 cube_00651_bnf.png
1 cub_01589_bnf.png
2 cup_00320_bnf.png
3 bow_00098_bnf.png
4 vas_00093_bnf.png
5 cylinder_02334_bnf.png
6 cylinder_01354_bnf.png
7 cube_00030_bnf.png
8 cub_00351_bnf.png
9 cube_00327_bnf.png
10 bla_00170_bnf.png
11 bow_00147_bnf.png
12 cylinder_02452_bnf.png
13 cub_00777_bnf.png
14 cub_01320_bnf.png
15 bla_00186_bnf.png
16 cup_00207_bnf.png
17 cube_00612_bnf.png
18 cylinder_01171_bnf.png
19 bow_00031_bnf.png
20 cylinder_01303_bnf.png
21 cylinder_01222_bnf.png
22 cube_01541_bnf.png
23 cube_00574_bnf.png
24 bla_00187_bnf.png
25 cub_01132_bnf.png
26 cylinder_01204_bnf.png
27 cube_00459_bnf.png
28 bow_00102_bnf.png
29 bla_00148_bnf.png
30 cube_01724_bnf.png
31 cube_00706_bnf.png
32 cube_00194_bnf.png
33 lov_00255_bnf.png
34 cub_00474_bnf.png
35 cup_00104_bnf.png
36 jui_00079_bnf.png
37 vas_00224_bnf.png
38 cube_00655_bnf.png
39 cube_00464_bnf.png
40 vas_00257_bnf.png
41 cub_00257_bnf.png
42 cylinder_01984_bnf.png
43 cube_00228_bnf.png
44 cup_00531_bnf.png
45 cylinder_

In [41]:
test_ov_all_np = np.array(test_ov_all)

ov_sum = test_ov_all_np.sum(0)
ov_count = np.sum(test_ov_all_np!=0,0)
ov = ov_sum/ov_count
print ov_sum/ov_count
print ov.mean()

#print(test_ov_all_np)
print('----')
print(test_ov_all_np.sum(0))
print(np.sum(test_ov_all_np!=0,0))
print('----')


[ 0.92177164  0.92020084  0.89353828  0.81096416]
0.886618731465
----
[ 152.0923204   149.07253662  155.47566063  135.43101543]
[165 162 174 167]
----


In [294]:
idx= 0
our_part_all = []
gt_part_all = []
bnf_part_all = []

our_part_label_all = []
gt_part_label_all = []
bnf_part_label_all = []

our_acc_all = []
bnf_acc_all = []

our_correct = np.zeros(len(classes))
our_all = np.zeros(len(classes))

bnf_correct = np.zeros(len(classes))
bnf_all = np.zeros(len(classes))

for imgname in bnf_list:
    realname = imgname[:imgname.index('_bnf')]   
    our_img = cv2.imread(our_path+realname+'_mask.png', 1)
    gt_img = cv2.imread(gt_path+realname+'.png', 1)
    bnf_img =  cv2.imread(bnf_path+realname+'_bnf.png', 1)
    
    h = gt_img.shape[0]
    w = gt_img.shape[1]
    
    if bnf_img is None:
        print("no bnf output")
        bnf_img =  np.zeros(gt_img.shape,np.uint8)
        
    idx = idx+1
    print(str(idx) + " " + imgname) 
    
    our_img = SetMask(our_img,h,w)
    bnf_img = SetMask(bnf_img,h,w)
    gt_img = SetMask(gt_img,h,w)
    
    correct, allpix = mask_level_acc(our_img,gt_img)
    bnfcorrect, allpix_bnf = mask_level_acc(bnf_img,gt_img)
    
    for k in range(0,len(classes)):
        our_correct[k] = our_correct[k]  + correct[k]
        our_all[k] = our_all[k]  + allpix[k]
        
        bnf_correct[k] = bnf_correct[k]  + bnfcorrect[k]
        bnf_all[k] = bnf_all[k]  + allpix_bnf[k]
    

our_acc_all = np.zeros(len(classes))
bnf_acc_all = np.zeros(len(classes))
for k in range(0,len(classes)):
    if our_all[k]!=0:
        our_acc_all[k] = our_correct[k]/our_all[k]  
    if bnf_all[k]!=0:
        bnf_acc_all[k] = bnf_correct[k]/bnf_all[k]  
    
    #our_part,our_part_label = ParseMask(our_img,h,w)
    #gt_part,gt_part_label = ParseMask(gt_img,h,w)
    #bnf_part,bnf_part_label = ParseMask(bnf_img,h,w)
    
    #our_part_all.append(our_part)
    #gt_part_all.append(gt_part)
    #bnf_part_all.append(bnf_part)    
    
    #our_part_label_all.append(our_part_label)
    #gt_part_label_all.append(gt_part_label)
    #bnf_part_label_all.append(bnf_part_label)    

1 bow_00098_bnf.png
2 vas_00093_bnf.png
3 cylinder_02334_bnf.png
4 cylinder_01354_bnf.png
5 cha_00078_bnf.png
6 cylinder_02452_bnf.png
7 bow_00031_bnf.png
8 cylinder_00821_bnf.png
9 cylinder_01204_bnf.png
10 lov_00255_bnf.png
11 jui_00079_bnf.png
12 vas_00224_bnf.png
13 vas_00257_bnf.png
14 cylinder_01984_bnf.png
15 cup_00531_bnf.png
16 jui_00093_bnf.png
17 vas_00164_bnf.png
18 vas_00062_bnf.png
19 jui_00528_bnf.png
20 cylinder_02692_bnf.png
21 jui_00283_bnf.png
22 cylinder_02539_bnf.png
23 cylinder_02389_bnf.png
24 vas_00174_bnf.png
25 cup_00476_bnf.png
26 bow_00166_bnf.png
27 cylinder_00035_bnf.png
28 cup_00341_bnf.png
29 cylinder_02905_bnf.png
30 cylinder_01258_bnf.png
31 cylinder_00964_bnf.png
32 cha_00088_bnf.png
33 jui_00497_bnf.png
34 cylinder_01648_bnf.png
35 cylinder_01657_bnf.png
36 cylinder_01146_bnf.png
37 cylinder_02923_bnf.png
38 cha_00167_bnf.png
39 cha_00195_bnf.png
40 cylinder_01530_bnf.png
41 cha_00222_bnf.png
42 cylinder_02400_bnf.png
43 cylinder_01582_bnf.png
44 cha

In [308]:
test_ov_all_np = np.array(test_ov_all)
print(test_ov_all_np)
print('----')
print(test_ov_all_np.mean(0))
print('----')
print(test_ov_all_np[:,2:5].mean())
print(test_ov_all_np[:,2:5])

[[ 0.          0.          0.88176456  0.8763497 ]
 [ 0.          0.          0.9282539   0.90315378]
 [ 0.          0.          0.9555955   0.75699043]
 [ 0.          0.          0.92861901  0.88144748]
 [ 0.          0.          0.94094231  0.83449612]
 [ 0.          0.          0.88058469  0.90459011]
 [ 0.          0.          0.96056026  0.93314169]
 [ 0.          0.          0.70314361  0.93320995]
 [ 0.          0.          0.89470009  0.        ]
 [ 0.          0.          0.90683492  0.93525091]
 [ 0.          0.          0.87731359  0.50531401]
 [ 0.          0.          0.88761826  0.6052735 ]
 [ 0.          0.          0.96343613  0.93180818]
 [ 0.          0.          0.92160689  0.68716844]
 [ 0.          0.          0.94496596  0.86115059]
 [ 0.          0.          0.90292706  0.85781782]
 [ 0.          0.          0.97052589  0.87349712]
 [ 0.          0.          0.94890433  0.9355388 ]
 [ 0.          0.          0.92892082  0.87171638]
 [ 0.          0.          0.91

In [219]:
our_ov_all = []
bnf_ov_all = []
for x in range(0,len(our_part_all)):
    print(x)
    our_ov = GetOverLap(our_part_all[x],gt_part_all[x],our_part_label_all[x],gt_part_label_all[x])
    bnf_ov = GetOverLap(bnf_part_all[x],gt_part_all[x],bnf_part_label_all[x],gt_part_label_all[x])
    
    print(our_ov)
    print(bnf_ov)
    
    our_ov_all.append(our_ov)
    bnf_ov_all.append(bnf_ov)

0
[ 0.          0.          0.90311688  0.89429713]
[ 0.          0.          0.85963618  0.80790085]
1
[ 0.          0.          0.9302037   0.91635741]
[ 0.          0.          0.91686912  0.86988636]
2
overlap==-1000
[ 0.          0.          0.96498597  0.80242723]
[ 0.          0.          0.96930574  0.8015914 ]
3
[ 0.          0.          0.93473992  0.87310059]
[ 0.          0.          0.91904787  0.77108249]
4
[ 0.          0.          0.95924744  0.80719483]
[ 0.          0.          0.95087465  0.58269618]


In [244]:
#our_ov_all.mean()
our_ov_all_np = np.array(our_ov_all)
bnf_ov_all_np = np.array(bnf_ov_all)

print(our_ov_all_np.mean(0))
print(bnf_ov_all_np.mean(0))

print(our_ov_all_np)
print(bnf_ov_all_np)

print(our_ov_all_np[0:5,2:5].mean())
print(bnf_ov_all_np[0:5,2:5].mean())


[ 0.          0.          0.93845878  0.85867544]
[ 0.          0.          0.92314671  0.76663146]
[[ 0.          0.          0.90311688  0.89429713]
 [ 0.          0.          0.9302037   0.91635741]
 [ 0.          0.          0.96498597  0.80242723]
 [ 0.          0.          0.93473992  0.87310059]
 [ 0.          0.          0.95924744  0.80719483]]
[[ 0.          0.          0.85963618  0.80790085]
 [ 0.          0.          0.91686912  0.86988636]
 [ 0.          0.          0.96930574  0.8015914 ]
 [ 0.          0.          0.91904787  0.77108249]
 [ 0.          0.          0.95087465  0.58269618]]


In [296]:
print(our_acc_all)
print(bnf_acc_all)

#our_acc_all = np.array(our_acc_all)
#bnf_acc_all = np.array(bnf_acc_all)

print(our_acc_all[2:4].mean())
print(bnf_acc_all[2:4].mean())

[ 0.          0.          0.96715469  0.98253832]
[ 0.          0.          0.95861711  0.98126067]
0.974846506669
0.969938890157
